In [ ]:
!pip install telethon pandas regex matplotlib

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 5.2 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=91847934fa9228dd73b470114a5fdb7740d567def110aa14711fa25137bbd26d
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [33]:
import os, re
import pandas as pd
from dotenv import load_dotenv
from telethon.sync import TelegramClient
import matplotlib.pyplot as plt

load_dotenv()   # if you’ll use a .env file later
print("✅ Environment ready")

✅ Environment ready


In [35]:
channels_df = pd.read_excel('/content/drive/MyDrive/channels_to_crawl.xlsx')
channels = channels_df.iloc[:,0].tolist()  # list of channels
print("Channels to fetch:", channels[:5], "...")  # preview first five


Channels to fetch: ['@nevacomputer', '@meneshayeofficial', '@ethio_brand_collection', '@Leyueqa', '@sinayelj'] ...


In [36]:
%env TELEGRAM_API_ID=22922331
%env TELEGRAM_API_HASH=7e39a6315d60fea1f076138b31713e73

API_ID   = os.getenv("TELEGRAM_API_ID")
API_HASH = os.getenv("TELEGRAM_API_HASH")
print("Loaded?", bool(API_ID and API_HASH))

env: TELEGRAM_API_ID=22922331
env: TELEGRAM_API_HASH=7e39a6315d60fea1f076138b31713e73
Loaded? True


In [37]:
import asyncio
from telethon import TelegramClient

def clean_text(text: str) -> str:
    return re.sub(r"[^\u1200-\u137F።፣\s]", "", text or "").strip()

async def fetch_all(channels, limit=500):
    async with TelegramClient("demo", API_ID, API_HASH) as client:
        records = []
        for ch in channels:
            print(f"Fetching {ch}…")
            async for m in client.iter_messages(ch, limit=limit):
                records.append({
                    "date":    m.date,
                    "sender":  m.sender_id,
                    "raw":     m.message,
                    "views":   m.views,
                    "channel": ch
                })
    df = pd.DataFrame(records)
    df["cleaned"] = df["raw"].apply(clean_text)
    return df

full_df = await fetch_all(channels[:5], limit=500)  # pick first 5 or all
print("Total messages:", len(full_df))
full_df.head()


Fetching @nevacomputer…
Fetching @meneshayeofficial…
Fetching @ethio_brand_collection…
Fetching @Leyueqa…
Fetching @sinayelj…
Total messages: 2500


,date,sender,raw,views,channel,cleaned
0,2025-06-21 06:32:27+00:00,-1001195361398,💻 Dell Precision Workstation\n\nA powerful per...,714.0,@nevacomputer,ለአጭር መልእክት\n ይደዉሉ
1,2025-06-21 06:32:27+00:00,-1001195361398,,714.0,@nevacomputer,
2,2025-06-21 06:32:27+00:00,-1001195361398,,714.0,@nevacomputer,
3,2025-06-21 06:32:27+00:00,-1001195361398,,616.0,@nevacomputer,
4,2025-06-11 13:56:52+00:00,-1001195361398,,1697.0,@nevacomputer,


In [38]:
output_path = "/content/drive/MyDrive/telegram_data.csv"  # or upload manually
full_df.to_csv(output_path, index=False)
print("✅ Saved to", output_path)

✅ Saved to /content/drive/MyDrive/telegram_data.csv
